In [ ]:
# TODO: test taht ps3.get_pdf_text(asset_url) is working as expected.


In [ ]:
# !pip uninstall -y foresight
#!pip install git+https://github.com/SAGE-3/next.git@dev#subdirectory=foresight

In [ ]:
# !pip install langchain_openai
# !pip install langchain

In [ ]:
PROMPTS_DIR = "src/prompts/"


In [1]:
import copy
keys_to_remove = ['_id', '_createdAt', '_createdBy', '_updatedAt', '_updatedBy',
    'touched', 'executeInfo', 'path', 'raised', 
    'title', 'roomId', 'boardId', 'lock', 'dragging', 'pinned', 'rotation'
]
def remove_keys_from_dict(app_dict, keys_to_remove):
    def _remove_keys_from_dict(d, keys_to_remove):
        if isinstance(d, dict):
            for key in list(d.keys()):  # Use list() to avoid 'dictionary changed size during iteration' error
                if key in keys_to_remove:
                    del d[key]
                else:
                    _remove_keys_from_dict(d[key], keys_to_remove)
        elif isinstance(d, list):
            for item in d:
                _remove_keys_from_dict(item, keys_to_remove)

    # Create a deep copy of the input dictionary
    app_dict_copy = copy.deepcopy(app_dict)
    # Remove keys from the copied dictionary
    _remove_keys_from_dict(app_dict_copy, keys_to_remove)
    return app_dict_copy


In [2]:
import sys
sys.path.append("/Users/mahdi/Documents/GitHub/next/foresight/")
# from src.config import config as conf, prod_type
from foresight.Sage3Sugar.pysage3 import PySage3
from src.prompts.prompt_loader import PromptLoader 
from langchain_openai import ChatOpenAI


In [6]:
prod_type= 'local'
conf = {'local': {'seer_server': 'http://127.0.0.1:5002',
  'jupyter_server': 'http://localhost:8888',
  'redis_server': 'localhost',
  'web_server': 'http://localhost:3333',
  'ws_server': 'ws://localhost:3333'}}
room_id = 'd6640933-7b9f-4536-a263-02a514ff092a'
board_id = '02fc98df-b4f1-470c-8516-7221c3095de7'
# app_id = '4a29b8bf-9e72-4366-8946-d808e2e3d465'

In [4]:
ps3 = PySage3(conf, prod_type)
cb = ps3.rooms['d6640933-7b9f-4536-a263-02a514ff092a'].boards['02fc98df-b4f1-470c-8516-7221c3095de7']

Configuring ps3 client ... 
I am here and app_info is {'_id': 'b18bd2d2-0840-40c9-bfc8-5970e5558c6c', '_createdAt': 1716848041212, '_createdBy': 'a70dd455-b081-4212-844b-5f3b509e42ee', '_updatedAt': 1716848068082, '_updatedBy': 'a70dd455-b081-4212-844b-5f3b509e42ee', 'data': {'title': 'Mahdi Bel-caid', 'roomId': 'd6640933-7b9f-4536-a263-02a514ff092a', 'boardId': '02fc98df-b4f1-470c-8516-7221c3095de7', 'position': {'x': 1500286, 'y': 1500222, 'z': 0}, 'size': {'width': 400, 'height': 420, 'depth': 0}, 'rotation': {'x': 0, 'y': 0, 'z': 0}, 'type': 'Stickie', 'state': {'text': 'test yellow', 'fontSize': 36, 'color': 'yellow', 'lock': False, 'executeInfo': {'executeFunc': '', 'params': {}}}, 'raised': False, 'dragging': False, 'pinned': False}}
I am here and app_info is {'_id': '96fa5557-16ee-4597-8708-bca17c6c7c02', '_createdAt': 1716848067418, '_createdBy': 'a70dd455-b081-4212-844b-5f3b509e42ee', '_updatedAt': 1716848082391, '_updatedBy': 'a70dd455-b081-4212-844b-5f3b509e42ee', 'data': {

In [ ]:
llm_4o = ChatOpenAI(temperature=0.1, model="gpt-4o")
#llm_4o.invoke("hello")

In [9]:
ps3.get_apps(room_id, board_id)

[{'_id': 'b18bd2d2-0840-40c9-bfc8-5970e5558c6c',
  '_createdAt': 1716848041212,
  '_createdBy': 'a70dd455-b081-4212-844b-5f3b509e42ee',
  '_updatedAt': 1716848068082,
  '_updatedBy': 'a70dd455-b081-4212-844b-5f3b509e42ee',
  'data': {'title': 'Mahdi Bel-caid',
   'roomId': 'd6640933-7b9f-4536-a263-02a514ff092a',
   'boardId': '02fc98df-b4f1-470c-8516-7221c3095de7',
   'position': {'x': 1500286, 'y': 1500222, 'z': 0},
   'size': {'width': 400, 'height': 420, 'depth': 0},
   'rotation': {'x': 0, 'y': 0, 'z': 0},
   'type': 'Stickie',
   'state': {'text': 'test yellow',
    'fontSize': 36,
    'color': 'yellow',
    'lock': False,
    'executeInfo': {'executeFunc': '', 'params': {}}},
   'raised': False,
   'dragging': False,
   'pinned': False}},
 {'_id': '96fa5557-16ee-4597-8708-bca17c6c7c02',
  '_createdAt': 1716848067418,
  '_createdBy': 'a70dd455-b081-4212-844b-5f3b509e42ee',
  '_updatedAt': 1716848082391,
  '_updatedBy': 'a70dd455-b081-4212-844b-5f3b509e42ee',
  'data': {'title': 'M

In [ ]:
example_prompt = PromptLoader.create_prompt(PROMPTS_DIR, "few_shot", "reframe_select_apps")
q = 'Can you delete all the red stickies with a greeting or hello'
# prompt_text = example_prompt.format(query=q)
(example_prompt | llm_4o).invoke({"query":q})

In [ ]:
all_apps = [remove_keys_from_dict(x, keys_to_remove) for x in ps3.get_apps()]
all_apps

In [ ]:
example_prompt = PromptLoader.create_prompt(PROMPTS_DIR, "few_shot", "select_apps")
q = 'Can you delete all the red stickies with a greeting or hello'
prompt_text = example_prompt.format(originalItems=all_apps, query=q)
print(prompt_text)
